# R CLUSTER TRENDS BY COUNTRY
This notebook generates clusters trends for a selected county (input)

In [2]:
from functions import *
DF, pL, bP, drugs=read_dataset('data/')

In [5]:
cnt='Argentina'
from sklearn import mixture
import statsmodels.api as sm
from functions import *

lS=[]
lA=[]
lT=[]
lI=[]
lEr=[]
lRt=[]
lRte=[]
lSt=[]
lSte=[]
lRv=[]
lRve=[]
lRI=[]
lSv=[]
lSve=[]
lSI=[]
lB=[]
#lE=[]
#lE2=[]
N=50
for sp in pL:
    for dr in drugs:
        print(sp,dr)
        BP=0
        BPbool=False
        if dr in bP[sp]:
            BP=bP[sp][dr][0]
            BPbool=True
        #GLOBAL TREND
        DFsp=DF[DF.Species==sp]
        DFc=pd.DataFrame({'Year': DFsp['Year'], 'MIC': DFsp[dr]})
        DFc=DFc.dropna()
        yL=get_key(DFc, 'Year')
        if len(yL)<2: continue
        x1=DFc['Year'].tolist()
        y1=DFc['MIC'].tolist()
        s1,i1,r1,p1,std1=stats.linregress(x1,y1)
        if p1<0.05:
            T=s1
            Er=std1
            I=i1
        else:
            T=0
            Er=0
            I=DFc['MIC'].mean()
        #LOOP FOR R CLUSTER
        Rtrend=[]
        Rvalue=[]
        Ryear=[]
        R2017=[]
        Strend=[]
        Svalue=[]
        Syear=[]
        S2017=[]
        i=-1
        while i<N:
            i=i+1
            #GET R CLUSTER
            try:#sometimes the Gaussian mixture does not converge, we need to generate a new smooth distribution
                pY,pM,sM=mic_dist(sp,dr,DF,0)
                mDFc, lDFc=extract_rclusternowrite(sp, dr, pY, pM, sM, DF)
            except:
                i=i-1
                continue
            #RCLUSTER TREND
            mDFc=mDFc[mDFc.Country==cnt]
            lDFc=lDFc[lDFc.Country==cnt]
            yLm=get_key(mDFc, 'Year') 
            if len(yLm)>=2:
                x1=mDFc['Year'].tolist()
                y1=mDFc['MIC'].tolist()
                s1,i1,r1,p1,std1=stats.linregress(x1,y1)
                if p1<0.05:
                    Rtrend.append(s1)#Rs=s1
                    mDFy=mDFc[mDFc.Year==2017]
                    if mDFy.empty:
                        R2017.append(s1*2017+i1)
                        E=True
                    else:
                        R2017.append(mDFy['MIC'].mean())
                        E=False
                else:
                    Rtrend.append(0.0)#Rs=0
                    R2017.append(mDFc['MIC'].mean())
                Rvalue.append(np.average(y1))
                Ryear.append(np.average(x1))
            #SCLUSTER TREND
            yLm=get_key(lDFc, 'Year') 
            if len(yLm)>=2:
                x1=lDFc['Year'].tolist()
                y1=lDFc['MIC'].tolist()
                s1,i1,r1,p1,std1=stats.linregress(x1,y1)
                if p1<0.05:
                    Strend.append(s1)#Rs=s1
                    lDFy=lDFc[lDFc.Year==2017]
                    if lDFy.empty:
                        S2017.append(s1*2017+i1)
                        E2=True
                    else:
                        S2017.append(lDFy['MIC'].mean())
                        E2=False
                else:
                    Strend.append(0.0)#Rs=0
                    S2017.append(lDFc['MIC'].mean())
                Svalue.append(np.mean(y1))
                Syear.append(np.mean(x1))
        #LISTS
        lS.append(sp)
        lA.append(dr)
        lT.append(T)
        lEr.append(Er)
        lI.append(I-BP)
        lRt.append(np.mean(Rtrend))
        lRte.append(np.std(Rtrend))
        lRv.append(np.mean(R2017)-BP)
        lRve.append(np.std(R2017))
        lRI.append(np.mean(Rvalue)-BP-np.mean(Rtrend)*np.mean(Ryear))
        lSt.append(np.mean(Strend))
        lSte.append(np.std(Strend))
        lSv.append(np.mean(S2017)-BP)
        lSve.append(np.std(S2017))
        lSI.append(np.mean(Svalue)-BP-np.mean(Strend)*np.mean(Syear))
        lB.append(BPbool)
        #lE.append(E)
        #lE2.append(E2)
        newDF=pd.DataFrame({'Species': lS, 'Antibiotic': lA, 'Trend':lT,
                    'Trenderror':lEr, 'Intercept': lI,
                    'Rtrend':lRt, 'Rtrenderror': lRte, 
                    'R2017': lRv, 'R2017error': lRve,
                    'RIntercept': lRI,
                    'Strend': lSt, 'Strenderror': lSte,
                    'S2017': lSv, 'S2017error': lSve,
                    'SIntercept': lSI, 'Breakpoint': lB})
        newDF.to_csv('results/Rcluster_country/resistant_cluster_trends_'+cnt+'.csv', index=False, float_format='%.6f')

Escherichia coli Doripenem
Escherichia coli Imipenem
Escherichia coli Meropenem
Klebsiella pneumoniae Doripenem
Klebsiella pneumoniae Imipenem
Klebsiella pneumoniae Meropenem
Pseudomonas aeruginosa Doripenem
Pseudomonas aeruginosa Imipenem
Pseudomonas aeruginosa Meropenem
Enterobacter cloacae Doripenem
Enterobacter cloacae Imipenem
Enterobacter cloacae Meropenem
Streptococcus pneumoniae Doripenem
Streptococcus pneumoniae Imipenem


/Users/carmen/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/carmen/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/carmen/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:233: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/carmen/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:194: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(
/Users/carmen/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Streptococcus pneumoniae Meropenem
Haemophilus influenzae Doripenem


/Users/carmen/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/carmen/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/carmen/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:233: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/carmen/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:194: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(
/Users/carmen/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Haemophilus influenzae Imipenem
Haemophilus influenzae Meropenem
Acinetobacter baumannii Doripenem
Acinetobacter baumannii Imipenem
Acinetobacter baumannii Meropenem
Streptococcus agalactiae Doripenem
Streptococcus agalactiae Imipenem


...

/Users/carmen/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/carmen/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/carmen/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:233: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/carmen/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:194: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(
/Users/carmen/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Streptococcus dysgalactiae Imipenem
Streptococcus dysgalactiae Meropenem


/Users/carmen/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/carmen/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/carmen/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:233: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/carmen/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:194: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(
/Users/carmen/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Clostridioides (Clostridium) difficile Doripenem
Clostridioides (Clostridium) difficile Imipenem
Clostridioides (Clostridium) difficile Meropenem


/Users/carmen/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/carmen/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/carmen/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:233: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/carmen/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:194: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(
/Users/carmen/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
